# IMPORT FUNCFIONS

In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import itertools
import pickle
import os
import random
from sklearn.model_selection import train_test_split

from sklearn.metrics import average_precision_score
from sklearn import metrics

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

from ge import LINE

# READ DATA

In [2]:
def get_edges_dict(traindata):
    if os.path.isfile("edgesdic.pkl"):
        f = open("edgesdic.pkl","rb")
        edges = pickle.load(f)
        f.close()
        return edges
    else:
        edges = dict()
        for edge in range(len(traindata)): 
            edges[(traindata.loc[[edge]]['id_1'].values[0],traindata.loc[[edge]]['id_2'].values[0])] = 1 
            edges[(traindata.loc[[edge]]['id_2'].values[0],traindata.loc[[edge]]['id_1'].values[0])] = 1  
        f = open("edgesdic.pkl","wb")
        pickle.dump(edges,f)
        f.close()
        return edges

def get_negative_edges(traindata,g):
    edges = get_edges_dict(traindata)
    # for missing edges.
    negative_edges = set([])
    maxNodenum = max(max(traindata['id_1']),max(traindata['id_2']))
    #產生與原圖同edge數目的negative edges
    while (len(negative_edges)<len(traindata)):
        node1=random.randint(0, maxNodenum) 
        node2=random.randint(0, maxNodenum) 
        tmp = edges.get((node1,node2),0) #edge不在graph裡標為0
        if tmp == 0 and node1!=node2 and g.has_node(node1) and g.has_node(node2): # if edge不在graph裡才要做處理
            negative_edges.add((node1,node2))
        else:
            continue
    return negative_edges

# MODELS

In [3]:
def combine_embedding(data,embeddings):
    i=0
    X = []
    for node1,node2 in data:
        X.append(np.concatenate((data[i],embeddings[int(node1)],embeddings[int(node2)])))
   # print(embeddings[str(int(data[0]))])
        i+=1
    return X

In [4]:
all_ap = []
all_auc = []

filename = 'cora'

for i in range(10):
    ori_df = pd.read_csv('data/'+filename+'.txt', header=None, sep=' ')
    ori_df.columns = ['id_1', 'id_2']

    ori_G = nx.from_pandas_edgelist(ori_df, 'id_1', 'id_2')
    #print(nx.info(ori_G))
    
    negative_edges = get_negative_edges(ori_df, ori_G)
    #print(len(negative_edges))

    train_df, test_df = train_test_split(ori_df, test_size=0.1)
    
    nodes_number = len(ori_G.nodes)
    graph_np = np.zeros((nodes_number, nodes_number))
    for i in range(train_df.shape[0]):
        graph_np[train_df.iloc[i, 0], train_df.iloc[i, 1]] = 1
        graph_np[train_df.iloc[i, 1], train_df.iloc[i, 0]] = 1

    train_G = nx.from_numpy_matrix(graph_np)
    #print(nx.info(train_G))
    
    df_neg = pd.DataFrame(list(negative_edges), columns=['id_1', 'id_2'])
    
    train_df = train_df.values
    test_pos_df = test_df.values
    df_neg = df_neg.values
    
    model = LINE(train_G, embedding_size=12, order='all')
    model.train(batch_size=128, epochs=50, verbose=2)
    
    LINE_embeddings = model.get_embeddings()
    
    train_df = combine_embedding(train_df, LINE_embeddings)
    test_pos_df = combine_embedding(test_pos_df, LINE_embeddings)
    df_neg = combine_embedding(df_neg, LINE_embeddings)
    
    X_train_neg, X_test_neg = train_test_split(df_neg, test_size=0.1)
    
    y_train_pos = np.ones(len(train_df))
    y_train_neg = np.zeros(len(X_train_neg))
    y_test_pos = np.ones(len(test_pos_df))
    y_test_neg = np.zeros(len(X_test_neg))
    
    X_train = np.concatenate((train_df, X_train_neg))
    y_train = np.concatenate((y_train_pos, y_train_neg))
    X_test = np.concatenate((test_pos_df, X_test_neg))
    y_test = np.concatenate((y_test_pos, y_test_neg))
    
    clf = RandomForestClassifier(n_estimators=400)
    clf.fit(X_train, y_train)
    
    predict_Y = clf.predict(X_test)
    
    ap = average_precision_score(y_test, predict_Y)
    print("AP： ", ap)
    fpr, tpr, _ = metrics.roc_curve(y_test, predict_Y, pos_label=1)
    auc = metrics.auc(fpr, tpr)
    print("AUC SCORE: ",auc)
    
    all_ap.append(ap)
    all_auc.append(auc)

print("####################### SUM UP ############################")
print("AP MEAN : ", np.array(all_ap).mean())
print("AP STD : ", np.array(all_ap).std())
print("AUC MEAN : ", np.array(all_auc).mean())
print("AUC STD : ", np.array(all_auc).std())

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Epoch 1/50
246/246 - 1s - loss: 1.3863 - first_order_loss: 0.6932 - second_order_loss: 0.6931
Epoch 2/50
246/246 - 0s - loss: 1.3852 - first_order_loss: 0.6929 - second_order_loss: 0.6924
Epoch 3/50
246/246 - 0s - loss: 1.3774 - first_order_loss: 0.6925 - second_order_loss: 0.6849
Epoch 4/50
246/246 - 0s - loss: 1.3408 - first_order_loss: 0.6917 - second_order_loss: 0.6491
Epoch 5/50
246/246 - 0s - loss: 1.2687 - first_order_loss: 0.6906 - second_order_loss: 0.5780
Epoch 6/50
246/246 - 0s - loss: 1.1880 - first_order_loss: 0.6889 - second_order_loss: 0.4991
Epoch 7/50
246/246 - 0s - loss: 1.1198 - first_order_loss: 0.6870 - second_order_loss: 0.4328
Epoch 8/50
246/246 - 0s - loss: 1.0682 - first_order_loss: 0.6846 - se

Epoch 26/50
246/246 - 0s - loss: 0.8457 - first_order_loss: 0.6425 - second_order_loss: 0.2032
Epoch 27/50
246/246 - 0s - loss: 0.8380 - first_order_loss: 0.6416 - second_order_loss: 0.1965
Epoch 28/50
246/246 - 0s - loss: 0.8332 - first_order_loss: 0.6395 - second_order_loss: 0.1938
Epoch 29/50
246/246 - 0s - loss: 0.8257 - first_order_loss: 0.6377 - second_order_loss: 0.1880
Epoch 30/50
246/246 - 0s - loss: 0.8170 - first_order_loss: 0.6358 - second_order_loss: 0.1812
Epoch 31/50
246/246 - 0s - loss: 0.8155 - first_order_loss: 0.6372 - second_order_loss: 0.1783
Epoch 32/50
246/246 - 0s - loss: 0.8113 - first_order_loss: 0.6367 - second_order_loss: 0.1747
Epoch 33/50
246/246 - 0s - loss: 0.8049 - first_order_loss: 0.6346 - second_order_loss: 0.1702
Epoch 34/50
246/246 - 0s - loss: 0.7987 - first_order_loss: 0.6337 - second_order_loss: 0.1650
Epoch 35/50
246/246 - 0s - loss: 0.7937 - first_order_loss: 0.6331 - second_order_loss: 0.1605
Epoch 36/50
246/246 - 0s - loss: 0.7861 - first_or

Epoch 12/50
245/245 - 0s - loss: 0.9655 - first_order_loss: 0.6725 - second_order_loss: 0.2930
Epoch 13/50
245/245 - 0s - loss: 0.9501 - first_order_loss: 0.6692 - second_order_loss: 0.2809
Epoch 14/50
245/245 - 0s - loss: 0.9394 - first_order_loss: 0.6667 - second_order_loss: 0.2728
Epoch 15/50
245/245 - 0s - loss: 0.9284 - first_order_loss: 0.6635 - second_order_loss: 0.2650
Epoch 16/50
245/245 - 0s - loss: 0.9168 - first_order_loss: 0.6610 - second_order_loss: 0.2558
Epoch 17/50
245/245 - 0s - loss: 0.9084 - first_order_loss: 0.6590 - second_order_loss: 0.2494
Epoch 18/50
245/245 - 0s - loss: 0.8966 - first_order_loss: 0.6565 - second_order_loss: 0.2401
Epoch 19/50
245/245 - 0s - loss: 0.8913 - first_order_loss: 0.6533 - second_order_loss: 0.2381
Epoch 20/50
245/245 - 0s - loss: 0.8834 - first_order_loss: 0.6530 - second_order_loss: 0.2304
Epoch 21/50
245/245 - 0s - loss: 0.8756 - first_order_loss: 0.6491 - second_order_loss: 0.2265
Epoch 22/50
245/245 - 0s - loss: 0.8705 - first_or

Epoch 48/50
246/246 - 1s - loss: 0.7436 - first_order_loss: 0.6268 - second_order_loss: 0.1168
Epoch 49/50
246/246 - 1s - loss: 0.7424 - first_order_loss: 0.6287 - second_order_loss: 0.1137
Epoch 50/50
246/246 - 1s - loss: 0.7354 - first_order_loss: 0.6263 - second_order_loss: 0.1091
AP：  0.7981718117408907
AUC SCORE:  0.84375
Epoch 1/50
245/245 - 1s - loss: 1.3863 - first_order_loss: 0.6931 - second_order_loss: 0.6931
Epoch 2/50
245/245 - 1s - loss: 1.3853 - first_order_loss: 0.6929 - second_order_loss: 0.6925
Epoch 3/50
245/245 - 1s - loss: 1.3784 - first_order_loss: 0.6924 - second_order_loss: 0.6860
Epoch 4/50
245/245 - 1s - loss: 1.3473 - first_order_loss: 0.6916 - second_order_loss: 0.6557
Epoch 5/50
245/245 - 1s - loss: 1.2811 - first_order_loss: 0.6904 - second_order_loss: 0.5907
Epoch 6/50
245/245 - 1s - loss: 1.1983 - first_order_loss: 0.6886 - second_order_loss: 0.5097
Epoch 7/50
245/245 - 1s - loss: 1.1293 - first_order_loss: 0.6862 - second_order_loss: 0.4431
Epoch 8/50
24

Epoch 34/50
245/245 - 1s - loss: 0.8009 - first_order_loss: 0.6344 - second_order_loss: 0.1665
Epoch 35/50
245/245 - 1s - loss: 0.7964 - first_order_loss: 0.6326 - second_order_loss: 0.1639
Epoch 36/50
245/245 - 1s - loss: 0.7930 - first_order_loss: 0.6365 - second_order_loss: 0.1565
Epoch 37/50
245/245 - 1s - loss: 0.7900 - first_order_loss: 0.6339 - second_order_loss: 0.1561
Epoch 38/50
245/245 - 1s - loss: 0.7835 - first_order_loss: 0.6312 - second_order_loss: 0.1523
Epoch 39/50
245/245 - 1s - loss: 0.7794 - first_order_loss: 0.6310 - second_order_loss: 0.1483
Epoch 40/50
245/245 - 1s - loss: 0.7736 - first_order_loss: 0.6312 - second_order_loss: 0.1424
Epoch 41/50
245/245 - 1s - loss: 0.7726 - first_order_loss: 0.6305 - second_order_loss: 0.1421
Epoch 42/50
245/245 - 1s - loss: 0.7663 - first_order_loss: 0.6321 - second_order_loss: 0.1342
Epoch 43/50
245/245 - 1s - loss: 0.7649 - first_order_loss: 0.6303 - second_order_loss: 0.1347
Epoch 44/50
245/245 - 1s - loss: 0.7577 - first_or

Epoch 20/50
245/245 - 1s - loss: 0.8816 - first_order_loss: 0.6528 - second_order_loss: 0.2288
Epoch 21/50
245/245 - 1s - loss: 0.8726 - first_order_loss: 0.6500 - second_order_loss: 0.2226
Epoch 22/50
245/245 - 1s - loss: 0.8703 - first_order_loss: 0.6488 - second_order_loss: 0.2215
Epoch 23/50
245/245 - 1s - loss: 0.8601 - first_order_loss: 0.6463 - second_order_loss: 0.2138
Epoch 24/50
245/245 - 1s - loss: 0.8533 - first_order_loss: 0.6479 - second_order_loss: 0.2054
Epoch 25/50
245/245 - 1s - loss: 0.8502 - first_order_loss: 0.6428 - second_order_loss: 0.2074
Epoch 26/50
245/245 - 1s - loss: 0.8444 - first_order_loss: 0.6434 - second_order_loss: 0.2010
Epoch 27/50
245/245 - 1s - loss: 0.8391 - first_order_loss: 0.6410 - second_order_loss: 0.1981
Epoch 28/50
245/245 - 1s - loss: 0.8299 - first_order_loss: 0.6389 - second_order_loss: 0.1910
Epoch 29/50
245/245 - 1s - loss: 0.8284 - first_order_loss: 0.6389 - second_order_loss: 0.1895
Epoch 30/50
245/245 - 1s - loss: 0.8201 - first_or